<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/BEST_PARAMETERS_FINDER_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner -q
!pip install ta -q
!pip install tensorflow -q

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
SOL_SQLITE_DB_PATH = '/content/gdrive/MyDrive/TradingBotLogs/ohlcv_data_SOL.db'
SOL_SQLITE_TABLE_NAME = 'solusd_1h_data'
SOL_MODEL_PATH = '/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v2_SOL.keras'



LDO_SQLITE_DB_PATH = '/content/gdrive/MyDrive/TradingBotLogs/ohlcv_data_LDO.db'
LDO_SQLITE_TABLE_NAME = 'ldousd_1h_data'
LDO_MODEL_PATH = '/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v2_LDO.keras'


TAO_SQLITE_DB_PATH = '/content/gdrive/MyDrive/TradingBotLogs/ohlcv_data_TAO.db'
TAO_SQLITE_TABLE_NAME = 'taousd_1h_data'
TAO_MODEL_PATH = '/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v2_TAO.keras'


ETH_SQLITE_DB_PATH = '/content/gdrive/MyDrive/TradingBotLogs/ohlcv_data.db'
ETH_SQLITE_TABLE_NAME = 'ethusd_1h_data'
ETH_MODEL_PATH = '/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v2.keras'



import sqlite3
import pandas as pd

db_configs = [
    {'db_path': SOL_SQLITE_DB_PATH, 'table_name': SOL_SQLITE_TABLE_NAME, 'symbol': 'SOL/USD'},
    {'db_path': LDO_SQLITE_DB_PATH, 'table_name': LDO_SQLITE_TABLE_NAME, 'symbol': 'LDO/USD'},
    {'db_path': TAO_SQLITE_DB_PATH, 'table_name': TAO_SQLITE_TABLE_NAME, 'symbol': 'TAO/USD'},
    {'db_path': ETH_SQLITE_DB_PATH, 'table_name': ETH_SQLITE_TABLE_NAME, 'symbol': 'ETH/USD'},
]

for config in db_configs:
    db_path = config['db_path']
    table_name = config['table_name']
    symbol = config['symbol']

    print(f"\n--- Data for {symbol} (Table: {table_name}, DB: {db_path}) ---")

    try:
        conn = sqlite3.connect(db_path)

        # Get the total number of rows to handle cases with less than 10 rows
        count_query = f"SELECT COUNT(*) FROM {table_name}"
        total_rows = pd.read_sql_query(count_query, conn).iloc[0, 0]
        print(f"Total rows: {total_rows}")

        if total_rows == 0:
            print("Table is empty.")
            conn.close()
            continue

        # Fetch first 5 rows
        head_query = f"SELECT * FROM {table_name} ORDER BY timestamp ASC LIMIT 5"
        df_head = pd.read_sql_query(head_query, conn)
        print("\nFirst 5 rows:")
        display(df_head)

        # Fetch last 5 rows, ensuring we don't overlap with the head if total rows are small
        tail_limit = min(5, total_rows)
        offset = max(0, total_rows - tail_limit)
        tail_query = f"SELECT * FROM {table_name} ORDER BY timestamp ASC LIMIT {tail_limit} OFFSET {offset}"
        df_tail = pd.read_sql_query(tail_query, conn)
        print("\nLast 5 rows:")
        display(df_tail)

        conn.close()

    except sqlite3.Error as e:
        print(f"Error accessing database {db_path} or table {table_name}: {e}")
    except Exception as e:
        print(f"An unexpected error occurred for {symbol}: {e}")

## BPF

In [3]:
!rm -rf /content/gdrive/MyDrive/TradingBotLogs/keras_tuner_dir/trading_param_tuning_LDO_USD

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import sqlite3
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
from ta.trend import MACD, CCIIndicator
from ta.volatility import BollingerBands, AverageTrueRange
from ta.volume import on_balance_volume
from ta.momentum import RSIIndicator, StochasticOscillator, WilliamsRIndicator
import logging
import keras_tuner as kt
import shutil

# Set up logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Helper Functions ---

def load_sqlite_data(symbol_config):
    """Load OHLCV data from SQLite database and preprocess it."""
    try:
        symbol = symbol_config['symbol']
        db_path = symbol_config['db_path']
        table_name = symbol_config['table_name']
        if not os.path.exists(db_path):
            raise FileNotFoundError(f"Database not found at {db_path}")
        conn = sqlite3.connect(db_path)
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql_query(query, conn, parse_dates=['timestamp'])
        conn.close()
        if df.empty:
            raise ValueError(f"No data in {db_path}/{table_name}")
        required_columns = ['open', 'high', 'low', 'close', 'volume', 'timestamp']
        if not all(col in df.columns for col in required_columns):
            raise ValueError(f"Missing required columns in {table_name}: {required_columns}")
        if df['timestamp'].dt.tz is None:
            df['timestamp'] = pd.to_datetime(df['timestamp']).dt.tz_localize('UTC').dt.tz_convert('America/New_York')
        else:
            df['timestamp'] = df['timestamp'].dt.tz_convert('America/New_York')
        df.set_index('timestamp', inplace=True)
        df = df[~df.index.duplicated(keep='last')]
        logging.info(f"Loaded {len(df)} rows from {db_path}/{table_name}")
        return df
    except Exception as e:
        logging.error(f"Failed to load data: {e}")
        raise

def calculate_technical_indicators(df):
    """Calculate technical indicators for the dataframe."""
    df = df.copy()
    df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].ffill()
    df['RSI'] = RSIIndicator(df['close'], window=14).rsi()
    macd = MACD(df['close'], window_slow=26, window_fast=12, window_sign=9)
    df['MACD'] = macd.macd()
    df['MACD_Signal'] = macd.macd_signal()
    bb = BollingerBands(df['close'], window=20, window_dev=2)
    df['BB_Upper'] = bb.bollinger_hband()
    df['BB_Lower'] = bb.bollinger_lband()
    df['OBV'] = on_balance_volume(df['close'], df['volume'])
    df['ATR'] = AverageTrueRange(df['high'], df['low'], df['close'], window=14).average_true_range()
    df['Williams_R'] = WilliamsRIndicator(high=df['high'], low=df['low'], close=df['close'], lbp=14).williams_r()
    stoch = StochasticOscillator(high=df['high'], low=df['low'], close=df['close'], window=14, smooth_window=3)
    df['Stoch_K'] = stoch.stoch()
    df['Stoch_D'] = stoch.stoch_signal()
    df['CCI'] = CCIIndicator(high=df['high'], low=df['low'], close=df['close'], window=14).cci()
    df = df.dropna()
    logging.info(f"After indicators, data has {len(df)} rows")
    return df

def prepare_backtest_data(df, look_back):
    """Prepare data for backtesting, caching calculated indicators."""
    if not hasattr(prepare_backtest_data, 'cached_df'):
        prepare_backtest_data.cached_df = calculate_technical_indicators(df)
    df = prepare_backtest_data.cached_df
    if len(df) < look_back:
        raise ValueError(f"Insufficient rows after indicators ({len(df)}, need {look_back})")
    return df, df['close'].copy(), df['ATR'].copy()

class PredictionAgent:
    def __init__(self, model_path, look_back=72, default_features=None):
        """Initialize prediction agent with a pre-trained model."""
        self.look_back = look_back
        self.default_features = default_features or [
            'open', 'high', 'low', 'close', 'volume', 'RSI', 'MACD', 'MACD_Signal',
            'BB_Upper', 'BB_Lower', 'OBV', 'ATR', 'Williams_R', 'Stoch_K', 'Stoch_D', 'CCI'
        ]
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.model = load_model(model_path)
        self.expected_features = self.model.input_shape[-1]
        logging.info(f"Model input shape: {self.model.input_shape}")
        self.features = self.default_features[:self.expected_features]
        if len(self.features) != self.expected_features:
            logging.warning(f"Feature mismatch: Model expects {self.expected_features}, using {len(self.features)}. Features: {self.features}")
        logging.info(f"Using features: {self.features}")
        self.predict_fn = tf.function(self.model, input_signature=[tf.TensorSpec(shape=(None, self.look_back, self.expected_features), dtype=tf.float32)])

    def preprocess(self, df):
        """Preprocess data for model prediction."""
        try:
            feature_data = []
            for f in self.features:
                if f in df.columns:
                    feature_data.append(df[f].values)
                else:
                    logging.warning(f"Feature {f} not in data, using zeros.")
                    feature_data.append(np.zeros(len(df)))
            data = np.column_stack(feature_data).astype(np.float32)
            scaled = self.scaler.fit_transform(data)
            X = scaled[-self.look_back:].reshape(1, self.look_back, self.expected_features)
            X = tf.convert_to_tensor(X, dtype=tf.float32)
            return X
        except Exception as e:
            logging.error(f"Preprocessing error: {e}")
            raise

    def execute(self, df):
        """Execute model prediction."""
        try:
            X = self.preprocess(df)
            pred_probs = self.predict_fn(X)
            pred_probs = np.asarray(pred_probs, dtype=np.float32)
            prob_sum = np.sum(pred_probs)
            if not np.isclose(prob_sum, 1.0, atol=0.01):
                logging.warning(f"Prediction probabilities sum to {prob_sum:.4f}, expected ~1.0")
            return pred_probs
        except Exception as e:
            logging.error(f"Prediction error: {e}")
            raise

def run_backtest_for_tuner(trade_params, prediction_agent, df_test, original_prices_test, original_atr_test, symbol="Generic", symbol_config=None, test_mode=False, trial_id=None):
    """Run backtest with trading parameters, storing trade details without per-trade logs."""
    original_log_level = logging.getLogger().level
    logging.getLogger().setLevel(logging.INFO)

    initial_capital = 2500.0
    capital = initial_capital
    qty = 0.0
    num_trades = 0
    in_position = False
    entry_price = 0.0
    equity_curve = [initial_capital]
    pred_classes = []
    trades = []

    look_back = prediction_agent.look_back
    atr_threshold = original_atr_test.rolling(window=1000, min_periods=1).quantile(0.25)

    logging.info(f"Starting backtest for {symbol} (Test Mode: {test_mode})")
    logging.info(f"Initial Capital: {initial_capital}, Look Back: {look_back}, Parameters: {trade_params}")

    for t in range(look_back, len(df_test)):
        current_timestamp = df_test.index[t]
        current_price = original_prices_test.iloc[t]
        atr = original_atr_test.iloc[t]

        window_df = df_test.iloc[max(0, t - look_back):t]
        if len(window_df) < look_back:
            equity_curve.append(capital)
            continue

        try:
            pred_probs = prediction_agent.execute(window_df)
            pred_class = np.argmax(pred_probs, axis=1)[0]
            confidence = pred_probs[0, pred_class]
            pred_classes.append(pred_class)
        except Exception as e:
            logging.warning(f"Prediction failed at step {t}: {e}")
            equity_curve.append(capital)
            continue

        is_atr_valid = atr >= atr_threshold.iloc[t] * symbol_config['atr_relax_factor']

        if not is_atr_valid:
            equity_curve.append(capital)
            continue

        if not in_position and pred_class == 1 and confidence >= trade_params['CONFIDENCE_THRESHOLD']:
            position_size_usd = capital * trade_params['MAX_POSITION_SIZE']
            if current_price > 0:
                qty = position_size_usd / current_price
            else:
                logging.warning(f"Current price is zero or negative at step {t}. Skipping trade.")
                equity_curve.append(capital)
                continue
            min_volume = symbol_config.get('min_volume', 0.001)
            if qty < min_volume:
                equity_curve.append(capital)
                continue
            capital -= position_size_usd
            entry_price = current_price
            in_position = True
            num_trades += 1
            trades.append({
                'trade_id': num_trades,
                'entry_time': str(current_timestamp),
                'entry_price': entry_price,
                'confidence': float(confidence),
                'quantity': qty
            })

        elif in_position:
            exit_reason = None
            exit_price = current_price
            tp_price = entry_price + atr * trade_params['ATR_MULTIPLIER_TP']
            sl_price = entry_price - atr * trade_params['ATR_MULTIPLIER_SL']

            if current_price >= tp_price:
                exit_reason = 'TP'
                exit_price = tp_price
            elif current_price <= sl_price:
                exit_reason = 'SL'
                exit_price = sl_price
            elif pred_class == 2 and confidence >= trade_params['CONFIDENCE_THRESHOLD']:
                exit_reason = 'Sell_Signal'

            if exit_reason:
                profit_or_loss = (exit_price - entry_price) * qty
                capital += (entry_price * qty) + profit_or_loss
                if trades and 'exit_time' not in trades[-1]:
                    trades[-1].update({
                        'exit_time': str(current_timestamp),
                        'exit_reason': exit_reason,
                        'exit_price': exit_price,
                        'profit_loss': profit_or_loss,
                        'capital': capital
                    })
                in_position = False
                qty = 0.0

        equity_curve.append(capital)

    if in_position:
        profit_or_loss = (original_prices_test.iloc[-1] - entry_price) * qty
        capital += original_prices_test.iloc[-1] * qty
        if trades and 'exit_time' not in trades[-1]:
            trades[-1].update({
                'exit_time': str(df_test.index[-1]),
                'exit_reason': 'Final_Close',
                'exit_price': original_prices_test.iloc[-1],
                'profit_loss': profit_or_loss,
                'capital': capital
            })

    total_return = ((capital - initial_capital) / initial_capital) * 100
    class_counts = np.bincount(pred_classes, minlength=3)
    tp_count = sum(1 for t in trades if t.get('exit_reason') == 'TP')
    sl_count = sum(1 for t in trades if t.get('exit_reason') == 'SL')
    sell_count = sum(1 for t in trades if t.get('exit_reason') == 'Sell_Signal')
    total_pl = sum(t.get('profit_loss', 0) for t in trades)
    logging.info(f"Backtest {'test' if test_mode else ''} Finished for {symbol}: Return {total_return:.2f}%, Trades {num_trades}, TP={tp_count}, SL={sl_count}, Sell={sell_count}, Total P/L {total_pl:.2f}, Classes: Buy={class_counts[1]}, Hold={class_counts[0]}, Sell={class_counts[2]}")

    trade_log_path = os.path.join(symbol_config['db_path'].rsplit('/', 1)[0], f"backtest_trades_{'test' if test_mode else f'trial_{trial_id}' if trial_id else 'final'}.json")
    try:
        with open(trade_log_path, 'w') as f:
            json.dump(trades, f, indent=4)
        logging.info(f"Saved trade log to {trade_log_path}")
    except Exception as e:
        logging.error(f"Error saving trade log: {e}")

    logging.getLogger().setLevel(original_log_level)
    return total_return, num_trades, equity_curve

# Corrected build_model function to work with kt.Hyperband
def build_model(hp, **kwargs):
    """
    Builds a dummy Keras model for Hyperband.
    This model's "training" is actually a backtest, and its "loss" is the negative total return.
    """
    trade_params = {
        'CONFIDENCE_THRESHOLD': hp.Float('confidence_threshold', min_value=0.001, max_value=0.05, step=0.001),
        'ATR_MULTIPLIER_TP': hp.Float('atr_multiplier_tp', min_value=1.0, max_value=5.0, step=0.5),
        'ATR_MULTIPLIER_SL': hp.Float('atr_multiplier_sl', min_value=0.5, max_value=3.0, step=0.5),
        'MAX_POSITION_SIZE': hp.Float('max_position_size', min_value=0.01, max_value=0.1, step=0.01)
    }

    # Use kwargs to pass necessary data for backtesting
    prediction_agent = kwargs['prediction_agent']
    df_test = kwargs['df_test']
    original_prices_test = kwargs['original_prices_test']
    original_atr_test = kwargs['original_atr_test']
    symbol_config = kwargs['symbol_config']

    total_return, num_trades, _ = run_backtest_for_tuner(
        trade_params, prediction_agent, df_test, original_prices_test, original_atr_test, symbol_config['symbol'], symbol_config
    )

    # We define a custom metric that Keras Tuner can track
    class CustomReturnMetric(tf.keras.metrics.Metric):
        def __init__(self, name='custom_return_metric', **kwargs):
            super().__init__(name=name, **kwargs)
            self.total_return_value = self.add_weight(name='total_return', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            # We don't use y_true or y_pred, we just set the value from the backtest
            self.total_return_value.assign(tf.constant(total_return, dtype=tf.float32))

        def result(self):
            return self.total_return_value

    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(1,)),
        tf.keras.layers.Dense(1)
    ])

    # We must use a loss that is independent of model weights and provides a gradient of 0.
    # We use a custom lambda layer that does nothing, ensuring no gradients are calculated.
    model.compile(
        optimizer='adam',
        loss='mse', # The loss doesn't matter since the model is not trained.
        metrics=[CustomReturnMetric()]
    )

    return model

# --- Main Execution ---
if __name__ == "__main__":
    # Mount Google Drive
    try:
        from google.colab import drive
        drive.mount('/content/gdrive', force_remount=True)
        logging.info("Google Drive mounted successfully")
    except Exception as e:
        logging.error(f"Failed to mount Google Drive: {e}")
        exit()

    symbol_config = {
        "symbol": "LDO/USD",
        "model_path": '/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v2_LDO.keras',
        "params": {"CONFIDENCE_THRESHOLD": 0.001, "ATR_MULTIPLIER_TP": 2.0, "ATR_MULTIPLIER_SL": 1.0, "MAX_POSITION_SIZE": 0.03},
        "db_path": '/content/gdrive/MyDrive/TradingBotLogs/ohlcv_data_LDO.db',
        "table_name": 'ldousd_1h_data',
        "min_volume": 0.001,
        "atr_relax_factor": 0.5
    }

    tuner_dir = '/content/gdrive/MyDrive/TradingBotLogs/keras_tuner_dir'
    project_name = f'trading_param_tuning_{symbol_config["symbol"].replace("/", "_")}'
    full_tuner_path = os.path.join(tuner_dir, project_name)

    # Clean up tuner directory
    if os.path.exists(full_tuner_path):
        try:
            shutil.rmtree(full_tuner_path)
            logging.info(f"Deleted tuner directory {full_tuner_path}")
        except Exception as e:
            logging.error(f"Error deleting tuner directory {full_tuner_path}: {e}")

    logging.info(f"Starting parameter optimization for {symbol_config['symbol']}")

    try:
        df = load_sqlite_data(symbol_config)
        df_test, original_prices_test, original_atr_test = prepare_backtest_data(df, 72)
        prediction_agent = PredictionAgent(symbol_config['model_path'])
    except Exception as e:
        logging.critical(f"Setup failed: {e}")
        exit()

    logging.info("Running standalone test backtest")
    default_trade_params = symbol_config['params']
    test_return, test_trades, test_equity_curve = run_backtest_for_tuner(
        default_trade_params, prediction_agent, df_test, original_prices_test, original_atr_test, symbol_config['symbol'], symbol_config, test_mode=True
    )
    logging.info(f"Test Backtest: Return {test_return:.2f}%, Trades {test_trades}")
    if test_trades == 0:
        logging.warning("No trades in test. Check model predictions or lower CONFIDENCE_THRESHOLD.")

    # Redefine build_model to pass the necessary backtest data
    def build_model_with_data(hp):
        return build_model(hp, prediction_agent=prediction_agent, df_test=df_test, original_prices_test=original_prices_test, original_atr_test=original_atr_test, symbol_config=symbol_config)

    tuner = kt.Hyperband(
        build_model_with_data,
        objective=kt.Objective('custom_return_metric', direction='max'),
        max_epochs=10,
        factor=3,
        directory=tuner_dir,
        project_name=project_name,
        overwrite=True
    )

    logging.info("Starting tuner search")
    try:
        # Dummy data to satisfy the Keras model.fit() call
        tuner.search(np.zeros((1, 1)), np.zeros((1, 1)), epochs=1)
        logging.info("Tuner search completed successfully")
    except Exception as e:
        logging.critical(f"Tuner search failed: {e}")
        exit()

    # The rest of the code for retrieving and logging results remains the same
    logging.info("Listing trials from tuner")
    saved_trial_results = []
    try:
        trials = tuner.oracle.get_best_trials(num_trials=1000)
        for i, trial in enumerate(trials):
            trial_id = trial.trial_id
            score = trial.score if trial.score is not None else -1000
            logging.info(f"Trial {i+1}: ID={trial_id}, Score={score:.2f}, Parameters={trial.hyperparameters.values}")

            trade_params = {
                'CONFIDENCE_THRESHOLD': trial.hyperparameters.get('confidence_threshold'),
                'ATR_MULTIPLIER_TP': trial.hyperparameters.get('atr_multiplier_tp'),
                'ATR_MULTIPLIER_SL': trial.hyperparameters.get('atr_multiplier_sl'),
                'MAX_POSITION_SIZE': trial.hyperparameters.get('max_position_size')
            }
            total_return, num_trades, _ = run_backtest_for_tuner(
                trade_params, prediction_agent, df_test, original_prices_test, original_atr_test, symbol_config['symbol'], symbol_config, trial_id=trial_id
            )
            trial_info = {
                'trial_id': trial_id,
                'parameters': trial.hyperparameters.values,
                'score': score,
                'total_return': total_return,
                'num_trades': num_trades,
                'status': 'completed'
            }
            trial_log_path = os.path.join(tuner_dir, project_name, trial_id, 'trial_log.json')
            os.makedirs(os.path.dirname(trial_log_path), exist_ok=True)
            with open(trial_log_path, 'w') as f:
                json.dump(trial_info, f, indent=4)
            saved_trial_results.append(trial_info)
    except Exception as e:
        logging.error(f"Error listing trials: {e}")

    all_trials_output_path = '/content/gdrive/MyDrive/TradingBotLogs/all_trials.json'
    try:
        with open(all_trials_output_path, 'w') as f:
            json.dump(saved_trial_results, f, indent=4)
        logging.info(f"Saved trial results to {all_trials_output_path}")
    except Exception as e:
        logging.error(f"Error saving trial results: {e}")

    logging.info("Retrieving best hyperparameters")
    try:
        best_trials = tuner.oracle.get_best_trials(num_trials=1)
        if not best_trials:
            logging.error("No successful trials found.")
            exit()
        best_hps = best_trials[0].hyperparameters
        best_trade_params = {
            'CONFIDENCE_THRESHOLD': best_hps.get('confidence_threshold'),
            'ATR_MULTIPLIER_TP': best_hps.get('atr_multiplier_tp'),
            'ATR_MULTIPLIER_SL': best_hps.get('atr_multiplier_sl'),
            'MAX_POSITION_SIZE': best_hps.get('max_position_size')
        }

        logging.info(f"Best Parameters for {symbol_config['symbol']}:")
        logging.info(f"  CONFIDENCE_THRESHOLD: {best_hps.get('confidence_threshold'):.4f}")
        logging.info(f"  ATR_MULTIPLIER_TP: {best_hps.get('atr_multiplier_tp'):.2f}")
        logging.info(f"  ATR_MULTIPLIER_SL: {best_hps.get('atr_multiplier_sl'):.2f}")
        logging.info(f"  MAX_POSITION_SIZE: {best_hps.get('max_position_size'):.4f}")

        final_return, final_trades, equity_curve = run_backtest_for_tuner(
            best_trade_params, prediction_agent, df_test, original_prices_test, original_atr_test, symbol_config['symbol'], symbol_config
        )
        logging.info(f"Final Total Return: {final_return:.2f}%")
        logging.info(f"Number of Trades: {final_trades}")

        best_params = {**best_trade_params, 'total_return': final_return, 'num_trades': final_trades}
        best_params_output_path = '/content/gdrive/MyDrive/TradingBotLogs/best_params_LDO.json'
        with open(best_params_output_path, 'w') as f:
            json.dump(best_params, f, indent=4)
        logging.info(f"Best parameters saved to {best_params_output_path}")
    except Exception as e:
        logging.critical(f"Error retrieving best hyperparameters: {e}")
        exit()

Trial 6 Complete [00h 04m 02s]
custom_return_metric: 6.4890546798706055

Best custom_return_metric So Far: 10.590460777282715
Total elapsed time: 00h 28m 19s

Search: Running Trial #7

Value             |Best Value So Far |Hyperparameter
0.021             |0.021             |confidence_threshold
3                 |3.5               |atr_multiplier_tp
2.5               |2                 |atr_multiplier_sl
0.09              |0.07              |max_position_size
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round



INFO:root:Starting backtest for LDO/USD (Test Mode: False)
INFO:root:Initial Capital: 2500.0, Look Back: 72, Parameters: {'CONFIDENCE_THRESHOLD': 0.021, 'ATR_MULTIPLIER_TP': 3.0, 'ATR_MULTIPLIER_SL': 2.5, 'MAX_POSITION_SIZE': 0.09}
